# 한국거래소(KRX) 분봉데이터 수집(크롤링)
## - 크롬드라이버 설치 : https://googlechromelabs.github.io/chrome-for-testing/

In [1]:
import sys
sys.version

'3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)]'

In [2]:
import numpy as np
import pandas as pd
from urllib.parse import urljoin
from urllib.parse import quote
from bs4 import BeautifulSoup
import requests as rq
import os
import csv
import time
from selenium import webdriver
from urllib.parse import urlencode
import ssl
import datetime
import sys
import json
import urllib
from urllib.request import Request, urlopen
import xmltodict
import openpyxl
import re
from selenium.webdriver.support import expected_conditions as EC
from urllib.request import urlretrieve
from PIL import ImageGrab
from PIL import Image
from selenium.common.exceptions import WebDriverException
from io import BytesIO
from selenium.webdriver.common.alert import Alert
import chardet
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

# 9시 개장과 10시 개장인 경우를 나눠서 수집
### 기본적인 원리는 동적 분봉 데이터 그래프에 마우스를 대면 나오는 가격데이터를 하나씩 가져오는 원리.
### 컴퓨터 사양에 따라 time.sleep 조정을 시간 텀을 좀 두면서 해야함.
### 2024년도 기준 1년간의 코스피 시총 5위 ~ 100위이며, 연도별 휴장일은 따로 데이터를 구해서 넣어줘야함. 직접 라벨링을 하거나 FnGuide를 이용하면 어렵지 않게 전처리 가능
### 화면 해상도에 따라 좌표 파라미터가 달라질 수 있으니 유의. (해당 코드는 전체 해상도 1920*1080)
### 수집 중에는 컴퓨터(특히 마우스)를 건드리지 않아야 함.

## 9시 개장 기준 하루를 크롤링 하는데 2분 30초 가량 걸리며, 1년을 하는데 9시간 30분.
## 즉, 한 종목을 1년 크롤링 하는데 9시간 30분 걸림

### *** 기타 정보 ***
### 웹 페이지 해상도 {'height': 600, 'width': 1920}
### 가장 왼쪽 그래프 X좌표 -876.5, 가장 오른쪽 그래프 X좌표 929
### 소수점 단위까지 정확한 수치는 아니며 직접 테스팅을 통해 좌표 움직임을 미세하게 조정했음. 컴퓨터 해상도에 따라 Open_nine과 Open_ten 함수에 있는 actions.move_to_element(box).move_by_offset()안의 값을 미세하게 조정하면서 찾으면 됨

# Q&A : egh962@naver.com, ghlee@kif.re.kr

## 수집을 위한 간단한 함수 생성

In [ ]:
def box_input(date):
    # 두 번째 날짜 박스 클릭
    second_date_box = driver.find_element(By.XPATH, f'/html/body/div[3]/div[2]/div[1]/div/input[2]') # search XPATH
    second_date_box.click()
    time.sleep(0.2)
    actions = ActionChains(driver)
    actions.double_click(second_date_box).perform()
    second_date_box.send_keys(Keys.BACKSPACE)
    time.sleep(0.3)
    second_date_box.send_keys(date) # 반드시 문자열로
    time.sleep(0.3)
    
    # 첫 번재 날짜 박스 클릭
    time.sleep(0.3)
    first_date_box = driver.find_element(By.XPATH, f'/html/body/div[3]/div[2]/div[1]/div/input[1]') # search XPATH
    first_date_box.click()
    time.sleep(0.2)
    actions = ActionChains(driver)
    actions.double_click(first_date_box).perform()
    first_date_box.send_keys(Keys.BACKSPACE)
    time.sleep(0.3)
    first_date_box.send_keys(date) # 반드시 문자열로
    time.sleep(0.3)
    

def Open_nine(): # 9시 개장
    data_basket  = []
    for i in range(391):
        actions.move_to_element(box).move_by_offset(-876.5+(i*(1802.5/389)), 100).perform() # 좌표 이동값
        time.sleep(0.15)
        price = driver.find_element(By.XPATH,"/html/body/div[4]/div/div").text # 시간 + 가격
    #    tic = driver.find_element(By.XPATH,"/html/body/div[4]/div/div/p[1]").text # 시간
        data_basket.append(price) # tic 점검
        time.sleep(0.2)
    return data_basket
    
def Open_ten(): # 10시 개장
    data_basket  = []
    for i in range(331):
        actions.move_to_element(box).move_by_offset(-876.5+(i*(1806.5/330)), 100).perform() # 좌표 이동값
        time.sleep(0.15)
        price = driver.find_element(By.XPATH,"/html/body/div[4]/div/div").text # 시간 + 가격
    #    tic = driver.find_element(By.XPATH,"/html/body/div[4]/div/div/p[1]").text # 시간
        data_basket.append(price) # tic 점검
        time.sleep(0.2)
    return data_basket

In [ ]:
form = pd.DataFrame({'ticker':[1],'date':[1], 'time':[1],'open':[1], 'high':[1],'low':[1],'close':[1],'volume':[1]})
date = pd.read_excel('분봉용파라미터.xlsx',sheet_name = 'nom_date')
ticker = pd.read_excel('분봉용파라미터.xlsx',sheet_name = 'ticker', dtype = object) 

for a in range(len(ticker)):
    chromedriver_path = './chromedriver.exe'
    service = Service(chromedriver_path)
    service.start()
    driver = webdriver.Chrome(service=service)
    url = f'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020203'
    driver.get(url)
    time.sleep(2)
    driver.maximize_window()
    time.sleep(0.5)
    
    # 종목 검색
    actions = ActionChains(driver)
    tic_box = driver.find_element(By.XPATH,f'/html/body/div[2]/section[2]/section/section/div/div/div[1]/div[1]/form/div[1]/div/table/tbody/tr/td/div/div/p/input')
    actions.double_click(tic_box).click(tic_box).perform() # 세번 클릭
    time.sleep(0.5)
    tic_box.send_keys(Keys.BACKSPACE)
    time.sleep(0.5)
    tic_box.send_keys(ticker['ticker'].iloc[a])
    time.sleep(0.5)
    driver.find_element(By.XPATH,f'/html/body/div[2]/section[2]/section/section/div/div/div[1]/div[1]/form/div[1]/div/table/tbody/tr/td/div/div/p/img').click()
    time.sleep(5)
    driver.find_element(By.XPATH,f'/html/body/div[2]/section[2]/section/section/div/div/div[1]/div[1]/form/div[1]/div/a').click() # 종목 조회 버튼 클릭
    time.sleep(2)
    
    # 차트 조회
    time.sleep(1)
    driver.find_element(By.XPATH,f'/html/body/div[2]/section[2]/section/section/div/div/div[1]/div[1]/div[1]/dl[2]/dt/ul/span').click()
    time.sleep(2)
    driver.switch_to.window(driver.window_handles[1])
    time.sleep(1)
    driver.maximize_window()
    time.sleep(1)
    for b in range(len(date)):
        # 날짜 입력 후 그래프 호출
        box_input(str(date['date'].iloc[b]))
        driver.find_element(By.XPATH,f'/html/body/div[3]/div[2]/div[2]/a').click()
        time.sleep(2)
        
        # 개장시간 확인
        box = driver.find_element(By.XPATH, '/html/body/div[4]')
        actions.move_to_element(box).move_by_offset(-876.5, 100).perform() # Leftmost bar location
        time.sleep(0.5)
        checking = driver.find_element(By.XPATH,"/html/body/div[4]/div/div/p[1]").text
        time.sleep(0.2)
        
        # 분봉 데이터 수집
        if checking == '09시 00분': # 개장 시간에 따른 수집 로직 결정
            daily_price =  Open_nine()
        else:
            daily_price =  Open_ten()
        
        # 분봉 데이터 데이터 프레임화
        for z in range(len(daily_price)):
            micro_daily_basket = daily_price[z].split('\n')
            price_form = pd.DataFrame({'ticker':ticker['ticker'].iloc[a],'date':[str(date['date'].iloc[b])], 'time':micro_daily_basket[0],
                  'open':micro_daily_basket[1].split(":", 1)[1].strip(), 'high':micro_daily_basket[2].split(":", 1)[1].strip(),
                                       'low':micro_daily_basket[3].split(":", 1)[1].strip(),'close':micro_daily_basket[4].split(":", 1)[1].strip(),
                                       'volume':micro_daily_basket[5].split(":", 1)[1].strip()})
            form = pd.concat([form,price_form]) # 데이터 붙이기 


# form 이라는 데이터 프레임에 최종 결과값들이 저장되며, for문의 range 파라미터나 자기만의 데이터를 넣어 조정 가능

In [ ]:
form.to_excel('Minute_data.xlsx')